In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from transformers import pipeline

In [ ]:
dataset=tfds.load("tiny_shakespeare",split='train',with_info=False)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/incomplete.3IWJQ1_1.0.0/tiny_shakespeare-train.tfrecord*.…

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/incomplete.3IWJQ1_1.0.0/tiny_shakespeare-validation.tfrec…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/incomplete.3IWJQ1_1.0.0/tiny_shakespeare-test.tfrecord*..…

Dataset tiny_shakespeare downloaded and prepared to /root/tensorflow_datasets/tiny_shakespeare/1.0.0. Subsequent calls will reuse this data.


In [ ]:
example=next(iter(dataset))
text=example["text"].numpy().decode("utf-8")

print(f"Длинна текста:{len(text)} символов")
print("Первые 500 символов:")
print(text[:500])

Длинна текста:1003854 символов
Первые 500 символов:
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [ ]:
vocab=sorted(set(text))
print(len(vocab))

65


In [ ]:
char_to_idx={ch:i for i,ch in enumerate(vocab)}
idx_to_char={i:ch for i,ch in enumerate(vocab)}

text_as_int=np.array([char_to_idx[c] for c in text])

print(f"Пример: First Citizen:-> {text_as_int[:13]}")


Пример: First Citizen:-> [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [ ]:
seq_length=100

example_per_epoch = len(text)//(seq_length+1)
print(example_per_epoch)

9939


In [ ]:
char_dataset=tf.data.Dataset.from_tensor_slices(text_as_int)

sequnces=char_dataset.batch(seq_length+1,drop_remainder=True)

def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset= sequnces.map(split_input_target)

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE= 10000

dataset=(
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.AUTOTUNE)
)



In [ ]:
vocab_size=len(vocab)
embedding_dim=256
rnn_units=1024
drop=0.2

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer="glorot_uniform"),
    tf.keras.layers.Dropout(drop),
    tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True),
    tf.keras.layers.Dropout(drop),
    tf.keras.layers.Dense(vocab_size)
])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer="adam",loss=tf.keras.losses.sparse_categorical_crossentropy)

In [ ]:
dir="/content/training_check"
prefix=dir+"/ckpt_{epoch}.weights.h5"

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=prefix,
    save_weights_only=True
)

In [ ]:
EPOCHS=30
history= model.fit(dataset,epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
134/155 ━━━━━━━━━━━━━━━━━━━━ 7:35 22s/step - loss: 4.4894

KeyboardInterrupt: 

In [ ]:
model.save("my_model.keras")
model.save("my_model.h5")

In [ ]:
loaded_model=tf.keras.models.load_model("my_model.keras")
loaded_model.save("my_model.h5")

In [ ]:
model.load_weights("/content/sample_data/chk1.h5")